In [44]:
import numpy as np
import qiskit
# class imports from QMSC
from qmsc.circuit import AnsatzCirc
from qmsc.backend import IBMQBackend

# Basic useage of AnsatzCirc
* Inherits from `QuantumCircuit` the `qiskit` class
    * This means, all same init + methods normally availible with `QuantumCircuit` still work

In [37]:
n = 3 # number of qubits
m = 2 # number of classical (measurement) registers
circ = AnsatzCirc(n, m, name = "name of circuit is an optional arg")
circ.x(0)
circ.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
          
q_2: ─────
          
c: 2/═════

## Also can add basic ansatz circuit elements
But in general, we recommend created an ansatz with the `FlexibleAnsatz` class and then converting it to a `AnsatzCirc` once it's ready

In [38]:
pvec = np.random.normal(size=15) % (2 * np.pi)
circ.add_arb_2q_gate(pvec, [0, 1])
circ.draw()

┌───┐     ┌───────────┐┌─────────────┐┌──────────────┐┌───┐»
q_0: ────┤ X ├─────┤ Rz(5.426) ├┤ Ry(0.65252) ├┤ Rz(0.047952) ├┤ X ├»
     ┌───┴───┴────┐└┬──────────┤└┬───────────┬┘└──────────────┘└─┬─┘»
q_1: ┤ Rz(1.2929) ├─┤ Ry(5.32) ├─┤ Rz(6.138) ├───────────────────■──»
     └────────────┘ └──────────┘ └───────────┘                      »
q_2: ───────────────────────────────────────────────────────────────»
                                                                    »
c: 2/═══════════════════════════════════════════════════════════════»
                                                                    »
«     ┌────────────┐                     ┌───┐ ┌────────────┐┌────────────┐»
«q_0: ┤ Rz(0.6722) ├──■──────────────────┤ X ├─┤ Rz(0.5944) ├┤ Ry(5.1353) ├»
«     ├────────────┤┌─┴─┐┌──────────────┐└─┬─┘┌┴────────────┤├────────────┤»
«q_1: ┤ Ry(4.5314) ├┤ X ├┤ Ry(0.070716) ├──■──┤ Rz(0.86764) ├┤ Ry(5.3171) ├»
«     └────────────┘└───┘└──────────────┘     └─────────────┘└────────────┘»
«q_2: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«c: 2/═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌───────────┐ 
«q_0: ┤ Rz(0.244) ├─
«     ├───────────┴┐
«q_1: ┤ Rz(5.9992) ├
«     └────────────┘
«q_2: ──────────────
«                   
«c: 2/══════════════
«

## Can ask for resulting unitary or statevector
This just combines methods availible to `qiskit` in an easier and more intutive way
* Both have optional method `reverse` which is a `bool` and defaults to `False`
    * When `False`, gives usual answer from Qiskit where basis vectors are ordered as $|q_3, q_2, q_1, q_0\rangle$ (least important bit to most important)
    * When `True` gives standard answer accepted by Quantum Info community, $|q_0, q_1, q_2, q_3\rangle$

In [39]:
# returns qiskit Operator object which has basic methods like `adjoint`
u = circ.get_unitary(reverse = False)
# check whether actually unitary
res1 = np.isclose(np.identity(2**3), np.matmul(u.adjoint(), u)).all()
print(f"udg * u = I is {res1}")
res2 = np.isclose(np.identity(2**3), np.matmul(u, u.adjoint())).all()
print(f"u * udg = I is {res2}")

udg * u = I is True
u * udg = I is True


In [40]:
# returns qiskit Statevector object which has basic methods with examples used below
qiskit_statevector = circ.get_statevector()
print("Qiskit version of statevector with reverse = False")
print("------------------------------------")
print(qiskit_statevector)
print("We can cast to a dictionary to make reading easier with `to_dict`")
print("------------------------------------")
print(qiskit_statevector.to_dict())
print("=======================================")
print("Normal version of statevector with reverse = True")
normal_statevector = circ.get_statevector(reverse = True)
print(normal_statevector)
print("We can cast to a dictionary to make reading easier with `to_dict`")
print(normal_statevector.to_dict())

Qiskit version of statevector with reverse = False
------------------------------------
Statevector([-0.19977205-0.40299918j,  0.6339046 -0.02862231j,
             -0.23707119+0.55533606j,  0.05572074-0.16529613j,
              0.        +0.j        ,  0.        +0.j        ,
              0.        -0.j        ,  0.        -0.j        ],
            dims=(2, 2, 2))
We can cast to a dictionary to make reading easier with `to_dict`
------------------------------------
{'000': (-0.19977205440658574-0.40299918464681944j), '001': (0.6339046032016977-0.02862230877354552j), '010': (-0.23707118655247145+0.5553360638002743j), '011': (0.055720736218206016-0.16529612589852946j)}
Normal version of statevector with reverse = True
Statevector([-0.19977205-0.40299918j,  0.        +0.j        ,
             -0.23707119+0.55533606j,  0.        -0.j        ,
              0.6339046 -0.02862231j,  0.        +0.j        ,
              0.05572074-0.16529613j,  0.        -0.j        ],
            dims=(2

In [41]:
normal_statevector.probabilities_dict()

{'000': 0.2023172165478291,
 '010': 0.36460089125057915,
 '100': 0.40265428251983004,
 '110': 0.030427609681761402}

In [42]:
normal_statevector.sample_counts(1000)

{'000': 210, '010': 345, '100': 424, '110': 21}

In [43]:
normal_statevector.sample_counts(1000)

{'000': 231, '010': 342, '100': 392, '110': 35}

#### We can also turn into a density matrix easily with `qiskit.quantum_info.DensityMatrix` class

In [46]:
pure_sigma = qiskit.quantum_info.DensityMatrix(normal_statevector)
print(pure_sigma.to_dict())

{'000|000': (0.20231721654782914+0j), '010|000': (-0.1764397829382905+0.2064801212353476j), '100|000': (-0.11510165778094635-0.2611809756597006j), '110|000': (0.055482758015026924-0.0554769579200564j), '000|010': (-0.1764397829382905-0.2064801212353476j), '010|010': (0.3646008912505791+0j), '100|010': (-0.1661755167332769+0.34524456246408997j), '110|010': (-0.1050046809687513-0.00824321437581917j), '000|100': (-0.11510165778094635+0.2611809756597006j), '010|100': (-0.1661755167332769-0.34524456246408997j), '100|100': (0.40265428251983004+0j), '110|100': (0.04005278793704692+0.10318711898135842j), '000|110': (0.055482758015026924+0.0554769579200564j), '010|110': (-0.1050046809687513+0.00824321437581917j), '100|110': (0.04005278793704692-0.10318711898135842j), '110|110': (0.0304276096817614+0j)}


In [47]:
print(pure_sigma.trace())

(0.9999999999999996+0j)


# Loading a backend and integrating with AnsatzCirc
To load in a backend, you need an IBM account where you can access the
* The `strname` of the the backend
* Your account's: `hub`
* `group`
* `project`
* `token`

Obviously, I will not actually run this part, as I don't want to share my account information...

Instead, I will fake run things and post the Markdown versions of results below.

### Loading the Backend
```
backend_name = "ibmq_example"
hub = "ibm-q-research-or-whatever"
group = "perhaps-your-uni"
project = "the-funds"
token = "your-token"
backend = IBMQBackend(backend, hub, group, project, token)
```
where the hub, group, project, and token info can be obtained by going to IBM quantum > account settings > providers. Then click the three vertical dots and select copy Qiskit provider code. If typing this is annoys you, you can hardcode your account as an __init__ option.

### Useful custom method: Printing backend properties
An important functon to be aware of when running experiments is
```
print(backend.get_readable_props_str())
```
which gives a human readable summary of the backend properties which can be saved to a txt file easily. For jakarta, for example, the result as of this tutorial are
```

Backend Properties
---
Experiment Date: 2022-07-07 12:06:44
Backend Name: ibmq_jakarta
Version: 1.0.34
Last Update Date: 2022-07-07 08:21:07-07:00

Gate Info
---
name, gate_error(), gate_length(ns)
id, 0.0003199501648000584, 35.55555555555556
id, 0.00020741584099435536, 35.55555555555556
id, 0.00020747484772500605, 35.55555555555556
id, 0.0002035022703507296, 35.55555555555556
id, 0.0010525762631233982, 35.55555555555556
id, 0.0002491638513245879, 35.55555555555556
id, 0.00020354933384976543, 35.55555555555556
rz, 0, 0
rz, 0, 0
rz, 0, 0
rz, 0, 0
rz, 0, 0
rz, 0, 0
rz, 0, 0
sx, 0.0003199501648000584, 35.55555555555556
sx, 0.00020741584099435536, 35.55555555555556
sx, 0.00020747484772500605, 35.55555555555556
sx, 0.0002035022703507296, 35.55555555555556
sx, 0.0010525762631233982, 35.55555555555556
sx, 0.0002491638513245879, 35.55555555555556
sx, 0.00020354933384976543, 35.55555555555556
x, 0.0003199501648000584, 35.55555555555556
x, 0.00020741584099435536, 35.55555555555556
x, 0.00020747484772500605, 35.55555555555556
x, 0.0002035022703507296, 35.55555555555556
x, 0.0010525762631233982, 35.55555555555556
x, 0.0002491638513245879, 35.55555555555556
x, 0.00020354933384976543, 35.55555555555556
cx, 0.017376502926994886, 504.88888888888886
cx, 0.017376502926994886, 540.4444444444445
cx, 0.007410021627299507, 384
cx, 0.007410021627299507, 419.55555555555554
cx, 0.006437586645274163, 277.3333333333333
cx, 0.006437586645274163, 312.88888888888886
cx, 0.008834249450477588, 291.55555555555554
cx, 0.008834249450477588, 327.1111111111111
cx, 0.00980800779616306, 248.88888888888889
cx, 0.00980800779616306, 284.44444444444446
cx, 0.007000849427907713, 234.66666666666666
cx, 0.007000849427907713, 270.22222222222223
reset, 7342.222222222222
reset, 7342.222222222222
reset, 7342.222222222222
reset, 7342.222222222222
reset, 7342.222222222222
reset, 7342.222222222222
reset, 7342.222222222222

Qubit Info
---
qubit, T1(us), T2(us), frequency(GHz), anharmonicity(GHz), readout_error(), prob_meas0_prep1(), prob_meas1_prep0(), readout_length(ns)
0, 179.27108623920228, 46.48443007226778, 5.236537333392189, -0.339883615358574, 0.03959999999999997, 0.05479999999999996, 0.0244, 5351.11111111111
1, 136.6107664533625, 28.38193214382445, 5.014431945688961, -0.3432005583724651, 0.035599999999999965, 0.03739999999999999, 0.0338, 5351.11111111111
2, 115.41307756584426, 26.005733303914802, 5.108468919342932, -0.3416150041672664, 0.024499999999999966, 0.0388, 0.010199999999999987, 5351.11111111111
3, 130.6752912614701, 43.29522100023257, 5.178135251335165, -0.3411171247904715, 0.017800000000000038, 0.0268, 0.00880000000000003, 5351.11111111111
4, 43.33845082911979, 50.386887681694404, 5.213062099531775, -0.3392533874360392, 0.1964999999999999, 0.29479999999999995, 0.0982, 5351.11111111111
5, 69.68705534461373, 49.38499263027378, 5.063262326256089, -0.3412893561600795, 0.040100000000000025, 0.050799999999999956, 0.0294, 5351.11111111111
6, 99.3803716167542, 23.117838725006226, 5.300667969846487, -0.3383638923290693, 0.049900000000000055, 0.0364, 0.06340000000000001, 5351.11111111111
```

### Linking backend to a circuit
By linking a baceknd to a circuit, we get access to useful methods...
```
circ = AnsatzCirc(3, 2, name = "link with backend example", ibmq_backend = backend)
t_circ = circ.get_transpiled_circ()
backend.submit_job(t_circ)
```